In [1]:
# %load /Users/hotbaby/github/firstcell.py
import warnings
warnings.filterwarnings('ignore')

import os
import sys
import copy
import tqdm
import math
import time
import heapq
import datetime
import itertools
import functools
import collections
import multiprocessing

import sklearn
import scipy as sp
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import StratifiedKFold, KFold, train_test_split

pd.set_option('display.max_columns', None)

%matplotlib inline


# Learning Pytorch with Examples

As it core, Pytorch provides two main fetures:

* **An n-dimensional Tensor, similar to numpy but can run on GPUs**
* **Automatic differentiation for building and training neural networks**

## Tensors

Numpy provides an n-dimensional array object, and many functions for manipulating these arrays. Numpy is a generic framework for scientific computing; It does not know anything about computation graphs, or deep learning, or gradients. However we can easily use numpy to fit a two-layer network to random data by manually implementing the forward and backward pass through the network using numpy operations:

In [2]:
import numpy as np

# N is batch size; D_in is input dimension;
# H is hidden dimension; D_out is output dimension.
N, D_in, H, D_out = 64, 1000, 100, 10

# create random input and output data
x = np.random.randn(N, D_in)
y = np.random.randn(N, D_out)

# Randomly initialzie weights
w1 = np.random.randn(D_in, H)
w2 = np.random.randn(H, D_out)

learning_rate = 1e-6

iterations = []
losses = []

for t in range(500):
    # Forward pass: compute predicted y
    h = x.dot(w1)
    h_relu = np.maximum(h, 0)
    y_pred = h_relu.dot(w2)
    
    # Compute and print loss
    loss = np.square(y_pred - y).sum()
    print(t, loss)
    iterations.append(t)
    losses.append(loss)
    
    # Backprop to compute gradients of w1 and w2 with respect to loss
    grad_y_pred = 2.0 * (y_pred - y)
    grad_w2 = h_relu.T.dot(grad_y_pred)
    grad_h_relu = grad_y_pred.dot(w2.T)
    grad_h = grad_h_relu.copy()
    grad_h[h<0] = 0
    grad_w1 = x.T.dot(grad_h)
    
    # Update weights
    w1 -= learning_rate * grad_w1
    w2 -= learning_rate * grad_w2

0 47254487.78302184
1 57632634.30867569
2 70083502.39781642
3 61262377.371271566
4 32106247.376402274
5 10286133.386738662
6 3452419.226282389
7 1882822.423605172
8 1388806.1616757181
9 1120993.0216320038
10 929507.3662668122
11 780367.4787751567
12 661051.5617531786
13 564558.6702336518
14 485410.29418667
15 419894.88769770745
16 365231.457064109
17 319273.7163654024
18 280365.0604957993
19 247268.41317763305
20 218953.3525911661
21 194590.62619221082
22 173529.17463271657
23 155232.8364107981
24 139265.32103266724
25 125301.10151550613
26 113031.0474902257
27 102235.45190400897
28 92674.73537988041
29 84186.46482139497
30 76627.36524077297
31 69878.00095717114
32 63834.34388463963
33 58415.62200122527
34 53539.66021066689
35 49139.04043908289
36 45163.49850348158
37 41569.23368168626
38 38308.03156245152
39 35347.59696573579
40 32651.44433658757
41 30193.13242412592
42 27946.964575960534
43 25893.43027150066
44 24013.29208724718
45 22290.09847725915
46 20707.648256002976
47 19252.125

444 8.318753428039377e-05
445 7.957473637496161e-05
446 7.61187979082564e-05
447 7.281400480672433e-05
448 6.965215989317759e-05
449 6.662825076954583e-05
450 6.373532619532386e-05
451 6.09691072626263e-05
452 5.8322287383319055e-05
453 5.579035393298421e-05
454 5.33690494587851e-05
455 5.105257403149085e-05
456 4.883645403232853e-05
457 4.671698669024884e-05
458 4.468938313773911e-05
459 4.275055704544017e-05
460 4.089594829781581e-05
461 3.912184287718585e-05
462 3.7424267242619784e-05
463 3.580041418831007e-05
464 3.424717809026152e-05
465 3.276136514168631e-05
466 3.134022862675503e-05
467 2.998080119425391e-05
468 2.8680738736021563e-05
469 2.7436948837227978e-05
470 2.6247050582385402e-05
471 2.51087945231683e-05
472 2.401985230664751e-05
473 2.29781217734968e-05
474 2.198185542877361e-05
475 2.1028877479355906e-05
476 2.0117042094663737e-05
477 1.9244858165489592e-05
478 1.84105781680935e-05
479 1.7612382606173035e-05
480 1.6848886479879993e-05
481 1.6118560375491544e-05
482 1.5

In [3]:
pd.Series(np.random.randn(100000)).hist

<bound method hist_series of 0       -0.282451
1       -1.084762
2       -0.030006
3       -0.348363
4        1.202684
           ...   
99995    2.383442
99996    1.249581
99997   -0.010868
99998   -1.039870
99999    0.205179
Length: 100000, dtype: float64>

### Pytorch: Tensors

Here we introduced the most fundamental PyTorch concept: the **Tensor**. A PyTorch Tensor is indentical to a numpy array: A Tensor is an n-dimensional array, and PyTorch provides many functions for operating on these Tensors, Behind the scenes, Tensors can keep track of a computational graph an gradients, but they're also useful as a generic tool for scientific omputing.

In [4]:
import torch


dtype = torch.float
device = torch.device('cpu')

# N is batch size; D_in is input dimension;
# H is hidden dimension; D_out is output dimension.
N, D_in, H, D_out = 64, 1000, 100, 10

# Create random input and output data
x = torch.randn(N, D_in, device=device, dtype=dtype)
y = torch.randn(N, D_out, device=device, dtype=dtype)

# Randomly initialize weights
w1 = torch.randn(D_in, H, device=device, dtype=dtype)
w2 = torch.randn(H, D_out, device=device, dtype=dtype)

learning_rate = 1e-6
for t in range(500):
    # Forward pass: compute predict y
    h = x.mm(w1)
    h_relu = h.clamp(min=0)
    y_pred = h_relu.mm(w2)
    
    # Compute and print loss
    loss = (y_pred - y).pow(2).sum().item()
    if t % 100 == 99:
        print(t, loss)
        
    # Backprop to compute gradient of w1 and w2 with respect to loss
    grad_y_pred = 2.0 * (y_pred - y)
    grad_w2 = h_relu.t().mm(grad_y_pred)
    grad_h_relu = grad_y_pred.mm(w2.t())
    grad_h = grad_h_relu.clone()
    grad_h[h<0] = 0
    grad_w1 = x.t().mm(grad_h)
    
    
    # Update weights using gradient descent
    w1 -= learning_rate * grad_w1
    w2 -= learning_rate * grad_w2

99 1147.4044189453125
199 27.145517349243164
299 1.4125854969024658
399 0.07966497540473938
499 0.004929259419441223


## Autograd

### PyTorch: Tensor and autograd

We can use `automatic differentiation` to automate the computation of backward passes in neural networks. The `autograd` package in PyTorch provides exactly this functionality. When using autograd, the forward pass of your network will define a `computation graph`; nodes in the graph will be Tensors, and edges will be functions that produce output Tensors from input Tensors. Backpropagating through this graph then allows you easiy compute gradients.

In [5]:
import torch


dtype = torch.float
device = torch.device('cpu')

# N is batch size; D_in is input dimension;
# H is hidden size, D_out is output dimension.
N, D_in, H, D_out = 64, 1000, 100, 10

# Create random Tensors to hold input and outputs.
# Settings requires_grad=False indicates that we do not need to compute gradients.
# with respect to these Tensors during the backward pass.
x = torch.randn(N, D_in, device=device, dtype=dtype)
y = torch.randn(N, D_out, device=device, dtype=dtype)

# Create random Tensors for weights.
w1 = torch.randn(D_in, H, device=device, dtype=dtype, requires_grad=True)
w2 = torch.randn(H, D_out, device=device, dtype=dtype, requires_grad=True)


learning_rate = 1e-6
for t in range(500):
    # Forward pass: compute predicted y using operations on Tensors;
    # these are exactly the same operations  we uesed to compute the
    # forward pass using Tensors, but we do not need to keep references
    # to intermadiate values since we are not implementing the backward
    # pass by hand.
    y_pred = x.mm(w1).clamp(min=0).mm(w2)
    
    # Compute and print loss using operations on Tensors.
    # Now loss is a Tensor of shape(1,)
    # loss.item() gets the scalar value held in the loss
    loss = (y_pred - y).pow(2).sum()
    if t % 100 == 99:
        print(t, loss.item())
        
    # Use autograd to compute the backward pass. This call will compute
    # the gradient of loss with respect to all Tensors with requires_grad=True
    # After this call w1.grad and w2.grad will be Tensors holding the
    # gradient of the loss with respect to w1 and w2 respectively.
    loss.backward()

    # Manully update weights using gradient descent. Wrap in torch.no_grad()
    # because weights have requires_grad=True, but we don't need to track
    # this in autograd.
    # An alternative way is to operate on weights.data and weights.grad.data.
    # Recall that tensor.data gives a tensor that shares the storage with
    # tensor, but doesn't track history.
    with torch.no_grad():
        w1 -= learning_rate * w1.grad
        w2 -= learning_rate * w2.grad
        
        # Manually zero the gradients after updating weights.
        w1.grad.zero_()
        w2.grad.zero_()


99 195.78231811523438
199 0.2987023890018463
299 0.0012851564679294825
399 7.390723476419225e-05
499 1.8942300812341273e-05


### PyTorch: Defining new autograd funtions

Under the hood, each primitive operator is really two functions that operate on Tensors. The **forward** function computes output Tensors from input Tensors. The **backward** function receives the gradient of the output Tensors with respect to some scalar value, and computes the gradient of the input Tensors with respect to that same scalar value.

In PyTorch we can easilly define our own autograd operator by defining a subclass of `torch.autograd.Function` and implementing the `forward` and `backward` functions. We can then use our new autograd operator by constructing an instance and calling it like a function, passing Tensors containing input data.

In this example we define our own custom autograd function for performing the ReLU nonlinearity, and use it to implement our two-layer network:

In [6]:
import torch


class MyReLU(torch.autograd.Function):

    @staticmethod
    def forward(ctx, input):
        """
        In the forward pass we receive a Tensor containing the input
        and return a Tensor containing the ouput. ctx is context object
        that can be used to stash information for backward computation.
        You can cache arbitrary objects for use in the backward pass using
        the ctx.save_for_backward method.
        """
        ctx.save_for_backward(input)
        return input.clamp(min=0)
    
    @staticmethod
    def backward(ctx, grad_output):
        """
        In the backward pass we receive a Tensor containing the gradient
        of the loss with respect to the output, and we need to compute
        the gradient of the loss with respect to the input.
        """
        input, = ctx.saved_tensors
        grad_input = grad_output.clone()
        grad_input[input<0] = 0
        return grad_input

In [7]:
dtype = torch.float
device = torch.device('cpu')

# N is batch size; D_in is input dimension;
# H is hidden dimension; D_out is output dimension.
N, D_in, H, D_out = 64, 1000, 100, 10

# Create random Tensor for weights.
x = torch.randn(N, D_in, dtype=dtype, device=device)
y = torch.randn(N, D_out, dtype=dtype, device=device)

# Create random Tensor for weigths
w1 = torch.randn(D_in, H, device=device, dtype=dtype, requires_grad=True)
w2 = torch.randn(H, D_out, device=device, dtype=dtype, requires_grad=True)

learning_rate = 1e-6
for t in range(500):
    # To apply our Function, we use Function.apply method.
    relu = MyReLU.apply
    
    # Forward pass: compute predicted y using operations; we 
    # compute ReLU using custom autograd operation.
    y_pred = relu(x.mm(w1)).mm(w2)
    
    # Compute and print loss
    loss = (y_pred - y).pow(2).sum()
    if t % 100 == 99:
        print(t, loss.item())
        
    # Use autograd to compute the backward pass.
    loss.backward()
    
    # Update weights using gradient descent
    with torch.no_grad():
        w1 -= learning_rate * w1.grad
        w2 -= learning_rate * w2.grad
        
        # Manually zero the gradient after updating weigths
        w1.grad.zero_()
        w2.grad.zero_()

99 486.11566162109375
199 2.026493787765503
299 0.014358902350068092
399 0.00029743570485152304
499 4.4454973249230534e-05


# nn module

## PyTorch: nn

Computational graphs and autograd are a very powerful paradigm for defining complex operations and automatically taking derivatives; however for large neural network raw autograd can be a bit too low-level.

When building neural networks we frequently think of arranging the computation into **layers**, some of which have **learnable parameters** which will be optimized during leanrning.

In PyTorch, the `nn` package defines a set of **Modules**, which are roughly equivalent to neural network layers. A Module receives input Tensors and computes output Tensors, but may also hold internal state such as Tensors containing learnable parameters.Then `nn` package also define a set of useful loss functions that are commonly used when training neural networks.

In this example, we use the `nn` package to implement our two-layer network:

In [8]:
import torch

# N is batch size; D_in is input dimension;
# H is hidden dimension; D_out is output dimension.
N, D_in, H, D_out = 64, 1000, 100, 10

# Create random Tensors to hold input and outputs.
x = torch.randn(N, D_in)
y = torch.randn(N, D_out)

# Use the nn package to define our model as a sequence of layers.
# nn.Sequential is a Module which contains other Module, and
# applies them in sequence to produce its output. Each Linear
# Module computes output from input using a linear function, and
# holds internal Tensors for its weight and bias.
model = torch.nn.Sequential(
    torch.nn.Linear(D_in, H),
    torch.nn.ReLU(),
    torch.nn.Linear(H, D_out),
)

# The nn package also contains definitions of popular loss function;
# in this case we will use Mean Square Error(MSE) as our loss function.
loss_fn = torch.nn.MSELoss(reduction='sum')

learning_rate = 1e-4
for t in range(500):
    y_pred = model(x)
    
    loss = loss_fn(y_pred, y)
    if t % 100 == 99:
        print(t, loss.item())
        
    # Zero the gradients before runing the backward pass.
    model.zero_grad()
    
    loss.backward()
    
    with torch.no_grad():
        for param in model.parameters():
            param -= learning_rate * param.grad

99 2.0404415130615234
199 0.025521380826830864
299 0.0006607453105971217
399 2.315367601113394e-05
499 9.224260679729923e-07


In [9]:
import torch

# N is batch size; D_in is input dimension;
# H is hidden dimension; D_out is output dimension.
N, D_in, H, D_out = 64, 1000, 100, 10

# Create random Tensors to hold input and outputs.
x = torch.randn(N, D_in)
y = torch.randn(N, D_out)

# Use the nn package to define our model as a sequence of layers.
# nn.Sequential is a Module which contains other Module, and
# applies them in sequence to produce its output. Each Linear
# Module computes output from input using a linear function, and
# holds internal Tensors for its weight and bias.
model = torch.nn.Sequential(
    torch.nn.Linear(D_in, H),
    torch.nn.ReLU(),
    torch.nn.Linear(H, D_out),
)

# The nn package also contains definitions of popular loss function;
# in this case we will use Mean Square Error(MSE) as our loss function.
loss_fn = torch.nn.MSELoss(reduction='sum')

learning_rate = 1e-4
for t in range(1000):
    y_pred = model(x)
    
    loss = loss_fn(y_pred, y)
    if t % 100 == 99:
        print(t, loss.item())
        
    # Zero the gradients before runing the backward pass.
    model.zero_grad()
    
    loss.backward()
    
    with torch.no_grad():
        for param in model.parameters():
            param -= learning_rate * param.grad

99 2.632279872894287
199 0.03160811960697174
299 0.0007909886189736426
399 2.767023215710651e-05
499 1.1569176194825559e-06
599 5.5285465094812025e-08
699 4.922322549560931e-09
799 1.4471459586218316e-09
899 7.052859962719538e-10
999 4.2898937180346763e-10


## PyTorch:optim

Up to this point we have updated the weights of our models by manually mutating the Tensors holding learnable parameters(with `torch.no_grad()` or `.data` to avoid tracking history in autograd). This is not huge burden for simple optimizaton algorithm like stochastic gradient descent, but in practice we often train neural networks using more sophisticated optimizers like AdaGrad, RMSProp, Adam, etc.

The `optim` package in PyTorch abstracts the idea of an optimization algorithm and provides implementations of commonly used optimization algorithms.

In [10]:
import torch

# N is batch size; D_in is input dimensin;
# H is hidden dimension; D_out is output dimension.
N, D_in, H, D_out = 64, 1000, 100, 10

# Create random Tensors to hold input and outputs.
x = torch.randn(N, D_in)
y = torch.randn(N, D_out)

# Use the nn package to define our model as a sequence of layers.
# nn.Sequential is a Module which contains other Module, and
# applies them in sequence to produce its output. Each Linear
# Module computes output from input using a linear function, and
# holds internal Tensors for its weight and bias.
model = torch.nn.Sequential(
    torch.nn.Linear(D_in, H),
    torch.nn.ReLU(),
    torch.nn.Linear(H, D_out),
)

# The nn package also contains definitions of popular loss function;
# in this case we will use Mean Square Error(MSE) as our loss function.
loss_fn = torch.nn.MSELoss(reduction='sum')

learning_rate = 1e-4
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
for t in range(500):
    y_pred = model(x)
    
    loss = loss_fn(y_pred, y)
    if t % 100 == 99:
        print(t, loss.item())
        
    # Zero the gradients before runing the backward pass.
    optimizer.zero_grad()
    
    loss.backward()
    
    optimizer.step()

99 58.797767639160156
199 1.1930639743804932
299 0.014867071062326431
399 0.00024547427892684937
499 2.2741896827938035e-06


## PyTorch:Custom nn Modules

Sometimes you will want to specifiy models that are more complex than a sequence of existing Modules; for these cases you can define your own Modules by subclassing `nn.Module` and defining a `forward` which receives input Tensors and produces output Tensors using other modules or other autograd operations on Tensors.

In [11]:
import torch

class TwoLayerNet(torch.nn.Module):
    def __init__(self, D_in, H, D_out):
        """
        In the custructor we instantiate two nn.Linear module
        and assign them as member variables.
        """
        super(TwoLayerNet, self).__init__()
        self.linear1 = torch.nn.Linear(D_in, H)
        self.linear2 = torch.nn.Linear(H, D_out)
        
    def forward(self, x):
        """
        In the forward function we accept a Tensor of input data 
        and we must return a Tensor of output data. We can use
        Modules defined in the constructor as well as arbitrary
        operations on Tensors.
        """
        h_relu = self.linear1(x).clamp(min=0)
        y_pred = self.linear2(h_relu)
        return y_pred
    
N, D_in, H, D_out = 64, 1000, 100, 10

# Create random Tensors to hold inputs and outputs
x = torch.randn(N, D_in)
y = torch.randn(N, D_out)

# Construct our model by instantiating the class defined above
model = TwoLayerNet(D_in, H, D_out)

criterion = torch.nn.MSELoss(reduction='sum')
optimizer = torch.optim.SGD(model.parameters(), lr=1e-4)

for t in range(500):
    y_pred = model(x)
    
    loss = criterion(y_pred, y)
    if t % 100 == 99:
        print(t, loss.item())
        
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

99 3.054292917251587
199 0.05412319302558899
299 0.001961934845894575
399 0.00010349514923291281
499 6.651531293755397e-06


## PyTorch: Control Flow + Weight Sharing

As an example of dynamic graphs and weight sharing, we implement a very strange model: a fully-connected ReLU network that on each forward pass chooses a random number between 1 and 4 and uses that many hidden layers, reusing the same weights multiple times to compute the innermost hidden layers.

In [12]:
import random
import torch


class DynamicNet(torch.nn.Module):
    def __init__(self, D_in, H, D_out):
        """
        In the constructor we construct three nn.Linear instance
        that we will use in the forward pass.
        """
        super(DynamicNet, self).__init__()
        self.input_linear = torch.nn.Linear(D_in, H)
        self.middle_linear = torch.nn.Linear(H, H)
        self.output_linear = torch.nn.Linear(H, D_out)
        
    def forward(self, x):
        """
        For the forward pass of the model, we randomly choose either
        0, 1, 2, or 3 and reuse the middle_linear Module that many
        times to compute hidden layer representations.
        
        Since each forward pass builds a dynamic computation graph,
        we can use normal Python control-flow operators like loop
        or conditional statements when defining the forward pass of
        the model.
        
        Here we also see that it is perfectly safe to reuse the same
        Module many times when defining a computational graph. This
        is a big improvement from Lua Torch, where each Module could
        be used only once.
        """
        h_relu = self.input_linear(x).clamp(min=0)
        for _ in range(random.randint(0, 3)):
            h_relu = self.middle_linear(h_relu).clamp(min=0)
        y_pred = self.output_linear(h_relu)
        return y_pred

N, D_in, H, D_out = 64, 1000, 100, 10

x = torch.randn(N, D_in)
y = torch.randn(N, D_out)

model = DynamicNet(D_in, H, D_out)
criterion = torch.nn.MSELoss(reduction='sum')
optimizer = torch.optim.SGD(model.parameters(), lr=1e-4, momentum=0.9)

for t in range(500):
    y_pred = model(x)
    loss = criterion(y_pred, y)
    if t % 100 == 99:
        print(t, loss.item())
        
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()


99 67.45323944091797
199 3.9014077186584473
299 0.9674086570739746
399 0.08855998516082764
499 10.204703330993652
